In [99]:
import pandas as pd
import sys
import os
from pathlib import Path
import sqlite3

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.utils import DBConnection
from src.config import DBConfig

In [2]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [3]:
pd.set_option('display.max_columns', 150)

In [4]:
df = pd.read_csv('umpire_data_all_multiseason.csv')

In [9]:
df_ump = df[['Date', 'Umpire', 'Home', 'Away', 'R [H]']].copy()

In [34]:
df_ump.columns = ['game_date', 'umpire', 'home_team', 'away_team']

In [13]:
df_ump = df_ump.drop('nan', axis=1)

In [20]:
df_ump = df_ump[~df_ump['date'].str.startswith('2015')]

In [35]:
df_ump.sort_values(by='game_date').head()

,game_date,umpire,home_team,away_team
20430,2016-04-03,Gerry Davis,KC,NYM
20429,2016-04-03,Mike Everitt,TB,TOR
20428,2016-04-03,Jerry Layne,PIT,STL
20419,2016-04-04,Mike Winters,BAL,MIN
20418,2016-04-04,Brian Gorman,MIL,SF


In [19]:
df_ump.dtypes

date      object
umpire    object
home      object
away      object
dtype: object

In [36]:
with NotebookDBConnection() as conn:
    df_ump.to_sql('umpire_data', conn, if_exists='replace', index=False)

In [112]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['statcast_pitchers', 'statcast_batters', 'team_mapping', 'pitcher_mapping', 'prediction_features', 'umpire_data', 'team_batting', 'mlb_api', 'pitcher_features', 'combined_features', 'batter_features', 'game_level_pitchers', 'game_level_batters', 'game_level_team_stats', 'advanced_opponent_game_features', 'team_season_features', 'train_combined_features', 'test_combined_features']


In [85]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM train_combined_features")
    print(cursor.fetchall())


[(76667,)]


In [98]:
with NotebookDBConnection() as conn:
    pitchers = pd.read_sql_query('''
    SELECT statcast_pitchers.*, umpire_data.umpire 
    FROM statcast_pitchers 
    JOIN umpire_data 
    ON statcast_pitchers.game_date = umpire_data.game_date
    AND statcast_pitchers.home_team = umpire_data.home_team
    AND statcast_pitchers.away_team = umpire_data.away_team
    ''', conn)
    pitchers.head()


In [103]:
pitchers.shape

(7788466, 116)

In [117]:
with NotebookDBConnection() as conn:
    df1 = pd.read_sql_query("SELECT * FROM prediction_features ORDER BY game_date DESC LIMIT 100", conn)
df1

,gamePk,game_date,pitcher_id,team_id,opponent_team_id,is_home,rolling_3g_k9,rolling_3g_k_pct,rolling_3g_swstr_pct,rolling_3g_velocity,rolling_3g_K_std,rolling_5g_k9,rolling_5g_k_pct,rolling_5g_swstr_pct,rolling_5g_velocity,rolling_5g_K_std,rolling_10g_k9,rolling_10g_k_pct,rolling_10g_swstr_pct,rolling_10g_velocity,rolling_10g_K_std,rolling_3g_fastball_pct,rolling_5g_fastball_pct,rolling_3g_breaking_pct,rolling_5g_breaking_pct,rolling_3g_offspeed_pct,rolling_5g_offspeed_pct,career_k9,career_k_pct,K_last_game,days_since_last_game,rest_days_4_less,rest_days_5,rest_days_6_more,is_month_3,is_month_4,is_month_5,is_month_6,is_month_7,is_month_8,is_month_9,is_month_10,recent_vs_career_k9,throws_right,lag_1_fastball_percent,lag_2_fastball_percent,lag_1_breaking_percent,lag_2_breaking_percent,lag_1_offspeed_percent,lag_2_offspeed_percent,rolling_3g_k_per_9_std_lag1,rolling_5g_k_per_9_std_lag1,rolling_10g_k_per_9_std_lag1,ewma_5g_k_per_9_lag1,rolling_3g_k_percent_std_lag1,rolling_5g_k_percent_std_lag1,rolling_10g_k_percent_std_lag1,ewma_5g_k_percent_lag1,k_percent_change_lag1,rolling_3g_swinging_strike_percent_std_lag1,rolling_5g_swinging_strike_percent_std_lag1,rolling_10g_swinging_strike_percent_std_lag1,ewma_5g_swinging_strike_percent_lag1,swinging_strike_percent_change_lag1,rolling_3g_avg_velocity_std_lag1,rolling_5g_avg_velocity_std_lag1,rolling_10g_avg_velocity_std_lag1,ewma_10g_avg_velocity_lag1,rolling_3g_fastball_percent_std_lag1,rolling_5g_fastball_percent_std_lag1,rolling_10g_fastball_percent_std_lag1,ewma_10g_fastball_percent_lag1,fastball_percent_change_lag1,rolling_3g_breaking_percent_std_lag1,rolling_5g_breaking_percent_std_lag1,rolling_10g_breaking_percent_std_lag1,ewma_10g_breaking_percent_lag1,breaking_percent_change_lag1,rolling_3g_offspeed_percent_std_lag1,rolling_5g_offspeed_percent_std_lag1,rolling_10g_offspeed_percent_std_lag1,ewma_10g_offspeed_percent_lag1,offspeed_percent_change_lag1,opp_base_team_k_percent,opp_base_team_bb_k_ratio,opp_base_team_zone_percent,opp_base_team_o_swing_percent,opp_base_team_z_contact_percent,opp_base_team_contact_percent,opp_base_team_swstr_percent,opp_base_team_wfb_c,opp_base_team_wsl_c,opp_base_team_wct_c,opp_base_team_wcb_c,opp_base_team_wch_c,opp_base_team_wsf_c
0,778348,2025-04-12,663978,142,116,1,7.615385,0.183333,0.100090,87.306052,2.081666,6.300000,0.147368,0.085823,87.067364,2.167948,7.448276,0.192308,0.094763,87.265356,2.211083,0.457672,0.462938,0.322574,0.300756,0.219754,0.236306,8.130112,0.200495,2.0,6.0,0.0,0.0,1.0,0,1,0,0,0,0,0,0,0.774897,1.0,0.494382,0.455556,0.325843,0.244444,0.179775,0.300000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,778347,2025-04-12,571945,138,143,1,6.187500,0.166667,0.070344,88.844536,0.577350,7.253731,0.183673,0.082947,88.678388,0.547723,7.328571,0.184466,0.068581,88.622194,1.229273,0.527809,0.525945,0.384637,0.389116,0.071482,0.060482,6.504049,0.165049,3.0,6.0,0.0,0.0,1.0,0,1,0,0,0,0,0,0,1.115264,1.0,0.475000,0.565217,0.412500,0.358696,0.087500,0.065217,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,778347,2025-04-12,650911,143,138,0,9.600000,0.246154,0.142738,90.259978,1.527525,10.253165,0.263158,0.137461,90.100078,1.414214,9.252809,0.247967,0.136207,89.943282,1.663330,0.483184,0.488245,0.160278,0.145944,0.356538,0.365811,8.139706,0.205514,7.0,6.0,0.0,0.0,1.0,0,1,0,0,0,0,0,0,1.259648,0.0,0.462366,0.465909,0.107527,0.181818,0.430108,0.352273,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,778355,2025-04-12,663855,137,147,0,9.000000,0.228571,0.095594,91.669754,2.886751,7.200000,0.177778,0.067883,92.325680,2.509980,6.612245,0.162162,0.080669,91.122466,1.751190,0.664304,0.720113,0.258962,0.258272,0.053544,0.053544,8.473846,0.203187,6.0,6.0,0.0,0.0,1.0,0,1,0,0,0,0,0,0,0.849673,1.0,0.722222,0.620690,0.260870,0.260870,0.041667,0.068966,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [119]:
df1.shape

(19, 96)

In [118]:
df1['rolling_5g_k_per_9_std_lag1'].unique()

array([0], dtype=int64)

In [83]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    df = pd.read_sql_query("SELECT * FROM combined_features WHERE umpire IS NULL ORDER BY game_date LIMIT 10 OFFSET 2500", conn)

df

,pitcher_id,player_name,game_date,game_pk,home_team,away_team,p_throws,season,strikeouts,batters_faced,total_pitches,avg_velocity,max_velocity,avg_spin_rate,avg_horizontal_break,avg_vertical_break,zone_percent,swinging_strike_percent,innings_pitched,k_per_9,k_percent,fastball_percent,breaking_percent,offspeed_percent,rolling_3g_k9,rolling_3g_k_pct,rolling_3g_swstr_pct,rolling_3g_velocity,rolling_3g_K_std,rolling_5g_k9,rolling_5g_k_pct,rolling_5g_swstr_pct,rolling_5g_velocity,rolling_5g_K_std,rolling_10g_k9,rolling_10g_k_pct,rolling_10g_swstr_pct,rolling_10g_velocity,rolling_10g_K_std,rolling_3g_fastball_pct,rolling_5g_fastball_pct,rolling_3g_breaking_pct,rolling_5g_breaking_pct,rolling_3g_offspeed_pct,rolling_5g_offspeed_pct,career_k9,career_k_pct,is_home,K_last_game,days_since_last_game,rest_days_4_less,rest_days_5,rest_days_6_more,game_month,is_month_3,is_month_4,is_month_5,is_month_6,is_month_7,is_month_8,is_month_9,is_month_10,recent_vs_career_k9,throws_right,lag_1_fastball_percent,lag_2_fastball_percent,lag_1_breaking_percent,lag_2_breaking_percent,lag_1_offspeed_percent,lag_2_offspeed_percent,inning,score_differential,is_close_game,is_playoff,rolling_3g_k_per_9_std_lag1,rolling_5g_k_per_9_std_lag1,rolling_10g_k_per_9_std_lag1,ewma_5g_k_per_9_lag1,rolling_3g_k_percent_std_lag1,rolling_5g_k_percent_std_lag1,rolling_10g_k_percent_std_lag1,ewma_5g_k_percent_lag1,k_percent_change_lag1,rolling_3g_swinging_strike_percent_std_lag1,rolling_5g_swinging_strike_percent_std_lag1,rolling_10g_swinging_strike_percent_std_lag1,ewma_5g_swinging_strike_percent_lag1,swinging_strike_percent_change_lag1,rolling_3g_avg_velocity_std_lag1,rolling_5g_avg_velocity_std_lag1,rolling_10g_avg_velocity_std_lag1,ewma_10g_avg_velocity_lag1,rolling_3g_fastball_percent_std_lag1,rolling_5g_fastball_percent_std_lag1,rolling_10g_fastball_percent_std_lag1,ewma_10g_fastball_percent_lag1,fastball_percent_change_lag1,rolling_3g_breaking_percent_std_lag1,rolling_5g_breaking_percent_std_lag1,rolling_10g_breaking_percent_std_lag1,ewma_10g_breaking_percent_lag1,breaking_percent_change_lag1,rolling_3g_offspeed_percent_std_lag1,rolling_5g_offspeed_percent_std_lag1,rolling_10g_offspeed_percent_std_lag1,ewma_10g_offspeed_percent_lag1,offspeed_percent_change_lag1,split,umpire
0,663765,"Woodford, Jake",2017-03-06,509787,MIN,STL,R,2017,0,6,9,88.939903,95.0,2237.0,-0.189537,0.671216,0.000000,0.000000,1.000000,0.000000,0.000000,0.577778,0.272727,0.147727,7.714286,0.200000,0.027778,88.238889,1.414214,6.480000,0.260870,0.051322,87.101317,1.914854,7.156627,0.285714,0.065099,88.369681,2.603417,0.666667,0.570410,0.222222,0.261834,0.111111,0.167756,0.000000,0.000000,0,0.0,5.0,0.0,1.0,0.0,3,1,0,0,0,0,0,0,0,inf,1.0,0.577778,0.578313,0.272727,0.272727,0.147727,0.147541,5,0,1,0,4.487559,4.679496,4.977887,8.735585,0.192450,0.217307,0.231774,0.387245,0.000000,0.049076,0.056021,0.067425,0.112759,0.000000,0.888540,0.997723,1.116884,88.987456,0.070806,0.079586,0.088049,0.578343,0.000000,0.060685,0.068328,0.076676,0.270045,0.000000,0.046111,0.052014,0.059604,0.134800,-0.000208,train,None
1,664285,"Valdez, Framber",2017-03-06,509684,HOU,BOS,L,2017,1,7,12,88.939903,95.0,2237.0,-0.189537,0.671216,0.000000,0.250000,1.666667,5.400000,0.142857,0.577778,0.272727,0.147727,10.800000,0.285714,0.254167,90.346667,1.414214,9.000000,0.235294,0.262711,91.111795,0.816497,6.750000,0.192308,0.166766,90.639816,1.054093,0.333333,0.397436,0.333333,0.358974,0.266667,0.210256,18.000000,0.500000,0,2.0,4.0,1.0,0.0,0.0,3,1,0,0,0,0,0,0,0,0.500000,0.0,0.577778,0.578313,0.272727,0.272727,0.147727,0.147541,5,0,1,0,4.487559,4.679496,4.977887,8.735585,0.192450,0.217307,0.231774,0.387245,0.000000,0.049076,0.056021,0.067425,0.112759,0.000000,0.888540,0.997723,1.116884,88.987456,0.070806,0.079586,0.088049,0.578343,0.000000,0.060685,0.068328,0.076676,0.270045,0.000000,0.046111,0.052014,0.059604,0.134800,-0.000208,train,None
2,282332,"Sabathia, CC",2017-03-07,509824,NYY,TB,L,2017,2,9,16,88.939903,95.0,2237.0,-0.189537,0.671216,

In [6]:
with NotebookDBConnection() as conn:
    df_pred = pd.read_sql_query("SELECT * FROM mlb_api", conn)

In [7]:
df_pred

,gamePk,game_date,home_team_id,home_team_name,home_team_abbr,away_team_id,away_team_name,away_team_abbr,home_probable_pitcher_id,home_probable_pitcher_name,away_probable_pitcher_id,away_probable_pitcher_name
0,778348,2025-04-12,142,Minnesota Twins,MIN,116,Detroit Tigers,DET,663978,Chris Paddack,695549,Jackson Jobe
1,778347,2025-04-12,138,St. Louis Cardinals,STL,143,Philadelphia Phillies,PHI,571945,Miles Mikolas,650911,Cristopher Sánchez
2,778355,2025-04-12,147,New York Yankees,NYY,137,San Francisco Giants,SF,701542,Will Warren,663855,Jordan Hicks
3,778346,2025-04-12,133,Athletics,ATH,121,New York Mets,NYM,669372,J.T. Ginn,656849,David Peterson
4,778362,2025-04-12,110,Baltimore Orioles,BAL,141,Toronto Blue Jays,TOR,608372,Tomoyuki Sugano,670102,Bowden Francis
5,778349,2025-04-12,145,Chicago White Sox,CWS,111,Boston Red Sox,BOS,527048,Martín Pérez,690916,Richard Fitts
6,778353,2025-04-12,139,Tampa Bay Rays,TB,144,Atlanta Braves,ATL,656876,Drew Rasmussen,700363,AJ Smith-Shawver
7,778354,2025-04-12,146,Miami Marlins,MIA,120,Washington Nationals,WSH,645261,Sandy Alcantara,592866,Trevor Williams
8,778359,2025-04-12,114,Cleveland Guardians,CLE,118,Kansas City Royals,KC,682847,Luis L. Ortiz,547179,Michael Lorenzen
9,778357,2025-04-12,113,Cincinnati Reds,CIN,134,Pittsburgh Pirates,PIT,671096,Andrew Abbott,571760,Andrew Heaney


In [122]:
with NotebookDBConnection() as conn:
    df_pitch = pd.read_sql_query("SELECT * FROM statcast_pitchers LIMIT 20000", conn)
    df_batter = pd.read_sql_query("SELECT * FROM statcast_batters LIMIT 20000", conn)
    df_team = pd.read_sql_query("SELECT * FROM team_batting LIMIT 20000", conn)

In [123]:
df_pitch.to_csv('statcast_pitcher_subset.csv')
df_batter.to_csv('statcast_batter_subset.csv')
df_team.to_csv('team_batting.csv')